# Download of the dataset

This notebook is used to to download the BacDive dataset. It is also available a .py version, useful to call the `download()` function externally.

In [1]:
import os
import sys
import glob
import pandas
import time
import fnmatch
import gzip
import shutil
import ftplib
import numpy as np

from ftplib import FTP, error_perm

There are four mainly functions:
 * `prepare`: from some xlsx (specified in ifolder) it can retrieve the names of the files to download.
 * `download_ncbi_from_ftp`: given the genomes, the url, the suffix of the file (fna or gff) and a path, it downloads the files.
 * `download_patric_from_ftp`: download other files in a slightly different way.
 * `unpack_ncbi`: unpack the downloaded files using gzip.
 * `prepare_aflp_silco`: run the AFLP software and generate the .wri files.

## Functions

In [2]:
def prepare():
    ifolder = '../Data/configuration/'
    files_name = sorted([os.path.basename(f).replace('.xlsx', '') for f in glob.glob(ifolder + "*.xlsx")])

    print("Preparing the download...")
    print("Custom download files:", f"\n{files_name}")
    
    if not os.path.exists('../Data/BacDive/'):
        os.makedirs('../Data/BacDive/')    
    
    frames = []

    for file_name in files_name:

        file = pandas.read_excel(open(ifolder+file_name+'.xlsx', 'rb'),header=None) 

        file.columns = file.loc[9,:]
        file.dropna(axis=0, how='all', inplace=True)
        file.drop(file[(file['strains.Culture collection no.'].str.contains('http', na=False)) | 
                       (file['strains.Culture collection no.'] == 'URL')].index, inplace = True)
        file.drop(file[file['strains.ID_strains'] == 'References'].index, inplace = True)
        file.drop(file.head(5).index,inplace=True)
        file.drop(file.tail(1).index,inplace=True)
        file.drop(index=file.index[0], axis=0, inplace=True)

        file.to_excel('../Data/BacDive/'+file_name+'.xlsx',index=None)

        file_50CH = file.loc[file['kit_api_50CHac.ID_reference'] > 0]
        file_50CH = file_50CH.iloc[:, np.r_[0:12,152:203]]
        frames.append(file_50CH)

    result_file_50CH = pandas.concat(frames)
    result_file_50CH.to_excel('../Data/BacDive/Genome_50CH_ac.xlsx',index=None)

    result_file_50CH = result_file_50CH.replace(['-', '+', '+/-'], [0,1,0])
    result_file_50CH.to_excel('../Data/BacDive/Genome_50CH_ac_0_1.xlsx',index=None)
    
    print("** Generated files Genome_50CH_ac.xlsx and Genome_50CH_ac_0_1.xlsx into Data/BacDive/ folder. \n")
    
    genome_ncbi = result_file_50CH.loc[result_file_50CH['sequence_genomes.Sequence database'] == 'ncbi']
    sequence_genomes_ncbi = list(set(genome_ncbi['sequence_genomes.Seq. accession number']))
    
    genome_patric = result_file_50CH.loc[result_file_50CH['sequence_genomes.Sequence database'] == 'patric']
    sequence_genomes_patric = list(set(genome_patric['sequence_genomes.Seq. accession number']))
    
    return genome_ncbi, sequence_genomes_ncbi, genome_patric, sequence_genomes_patric 



def unpack_ncbi(folder_ncbi, files_name_ncbi, file_type):
    print(f"\nUnzip files in {folder_ncbi}...")
    decompress_folder = os.path.join(folder_ncbi, 'Decompress')
    
    if not os.path.exists(decompress_folder):
        os.makedirs(decompress_folder)

    for file in files_name_ncbi:
        input_file_path = os.path.join(folder_ncbi, file + file_type + '.gz')
        output_file_path = os.path.join(decompress_folder, file + file_type)

        with gzip.open(input_file_path, 'rb') as f_in:
            with open(output_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                
    print("Done!")

In [3]:
def download_ncbi_from_ftp(sequence_genomes, url_prefix, file_suffix, download_path):
    tic = time.perf_counter()

    ftp = ftplib.FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()
    
    if not os.path.exists(download_path):
        os.makedirs(download_path)
        
    for sequence in sequence_genomes:
                
        toc = time.perf_counter()

        if toc - tic >= 200:
            ftp.quit()
            ftp = ftplib.FTP('ftp.ncbi.nlm.nih.gov')
            ftp.login()

            tic = time.perf_counter()

        url = url_prefix

        if sequence[0:3] == 'GCA':
            for index in range(4, len(sequence), 3):
                url = url + sequence[index:(index + 3)] + '/'

        ftp.cwd(url)
        name = next(iter(ftp.nlst()))
        ftp.cwd(name)

        files = ftp.nlst()
                        
        if not any(file_suffix in string for string in files):
            print(f'  File {sequence} not found in GCA. Trying to find the .gff in GCF folder.')
            url = url[:12] + 'GCF' + url[15:]
            
            ftp.cwd('/.')
            try:
                ftp.cwd(url)
            except error_perm as e:
                if "550" in str(e):
                    print("")
                    print("="*80)
                    print(f"{url}: No such file or directory")
                    print("="*80,"\n")     
                else:
                    return " other error."
                    
            name = next(iter(ftp.nlst()))
            ftp.cwd(name)

            files = ftp.nlst()         
            
        # Download files
        for file in files:

            if fnmatch.fnmatch(file, name + file_suffix):  # To download specific files.

                if file[0:3] == 'GCF': # Change file name from GCF_* to GCA_*.
                    file_path = os.path.join(download_path, 'GCA' + file[3:])
                else:
                    file_path = os.path.join(download_path, file)
                

                if not os.path.exists(file_path):

                    try:
                        ftp.retrbinary("RETR " + file, open(file_path, 'wb').write)

                    except EOFError:  # To avoid EOF errors.
                        pass
                else:
                    print(f"  File already exists: {file_path}")            
                

        ftp.cwd('/.')

    ftp.quit()
    
    print(f"Files downloaded in {download_path}")

In [4]:
def download_patric_from_ftp(sequence_genomes, url_prefix, file_suffix, download_path):
    tic = time.perf_counter()

    ftp = ftplib.FTP('ftp.bvbrc.org')
    ftp.login()
    
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    # Unable to download entire sequence: "no file or directory" error when searching for data online.
    sequence_error = [525325.5, 1589.1, 1589.9, 1226298.1, 'GCA_001475175', 219334.1, 391904.2, 1358.133, 51663.5, 1423723.1, 273384.3, 518634.2]
    
    for sequence in sequence_genomes:

        toc = time.perf_counter()

        if toc - tic >= 200:
            ftp.quit()
            ftp = ftplib.FTP('ftp.bvbrc.org')
            ftp.login()

            tic = time.perf_counter()

        if sequence in sequence_error:
            print(f'  *** Strain {sequence} not downloaded: file not found online. ***\n')

        else:
            url = url_prefix + str(sequence) + '/'
            ftp.cwd(url)

            file_name = str(sequence) + file_suffix

            files = ftp.nlst()

            # Download files
            for file in files:
                if fnmatch.fnmatch(file, file_name):  # To download specific files.
                    file_path = os.path.join(download_path, file)
                    
                    if not os.path.exists(file_path):

                        try:
                            ftp.retrbinary("RETR " + file, open(file_path, 'wb').write)

                        except EOFError:  # To avoid EOF errors.
                            pass
                    else:
                        print(f"  File already exists: {file_path}")

            ftp.cwd('/.')

    ftp.quit()
  
    print(f"Files downloaded in {download_path}\n")

## Download files

In [5]:
def download(genome_ncbi, sequence_genomes_ncbi, genome_patric, sequence_genomes_patric):
    url_prefix = 'genomes/all/GCA/'
    file_suffix_fna = '_genomic.fna.gz'
    download_path_fna = '../Data/BacDive/ncbi/'
    file_type_fna = '.fna'

    file_suffix_gff = '_genomic.gff.gz'
    download_path_gff = '../Data/BacDive/gff/ncbi/'
    file_type_gff = '.gff'

    patric_url_prefix_fna = 'genomes/'
    patric_file_suffix_fna = '.fna'
    patric_download_path_fna = '../Data/BacDive/Patric/'

    patric_url_prefix_gff = 'genomes/'
    patric_file_suffix_gff = '.PATRIC.gff'
    patric_download_path_gff = '../Data/BacDive/gff/Patric/'

    paths = pandas.Series([download_path_fna, download_path_gff, patric_download_path_fna, patric_download_path_gff])

    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)


    try:
        print("\nDownloading ncbi fna files...")
        download_ncbi_from_ftp(sequence_genomes_ncbi, url_prefix, file_suffix_fna, download_path_fna)

        print("\nDownloading ncbi gff files... (Problemi)")
        download_ncbi_from_ftp(sequence_genomes_ncbi, url_prefix, file_suffix_gff, download_path_gff)
        
        print("\nDownloading Patric fna files...")
        download_patric_from_ftp(sequence_genomes_patric, patric_url_prefix_fna, patric_file_suffix_fna, patric_download_path_fna)
        
        print("\nDownloading Patric gff files...")
        download_patric_from_ftp(sequence_genomes_patric, patric_url_prefix_gff, patric_file_suffix_gff, patric_download_path_gff)

    except Exception as e:
        print("="*80)
        print(f"Errore: {e}")
        print("="*80)
        print('\n')
        
        
    files_name_ncbi_fna = sorted([f.replace('.fna.gz', '').replace(download_path_fna, '').replace(download_path_fna, '') for f in glob.glob(download_path_fna + "*.fna.gz")])
    files_name_ncbi_gff = sorted([f.replace('.gff.gz', '').replace(download_path_gff, '').replace(download_path_gff, '') for f in glob.glob(download_path_gff + "*.gff.gz")])

    unpack_ncbi(download_path_fna, files_name_ncbi_fna, file_type_fna)
    unpack_ncbi(download_path_gff, files_name_ncbi_gff, file_type_gff)

In [23]:
genome_ncbi, sequence_genomes_ncbi, genome_patric, sequence_genomes_patric = prepare() # Prepare all the files to download

download(genome_ncbi, sequence_genomes_ncbi, genome_patric, sequence_genomes_patric)



  File already exists: ../Data/BacDive/gff/ncbi/GCA_007992275.1_ASM799227v1_genomic.gff.gz
  File already exists: ../Data/BacDive/gff/ncbi/GCA_001025155.1_ASM102515v1_genomic.gff.gz
  File already exists: ../Data/BacDive/gff/ncbi/GCA_019753765.1_ASM1975376v1_genomic.gff.gz
  File already exists: ../Data/BacDive/gff/ncbi/GCA_009764355.1_ASM976435v1_genomic.gff.gz
  File already exists: ../Data/BacDive/gff/ncbi/GCA_018314255.1_ASM1831425v1_genomic.gff.gz
  File GCA_010120595 not found in GCA. Trying to find the .gff in GCF folder.
  File already exists: ../Data/BacDive/gff/ncbi/GCA_010120595.1_ASM1012059v1_genomic.gff.gz
  File GCA_000374385 not found in GCA. Trying to find the .gff in GCF folder.
  File already exists: ../Data/BacDive/gff/ncbi/GCA_000374385.1_ASM37438v1_genomic.gff.gz
  File already exists: ../Data/BacDive/gff/ncbi/GCA_001025175.1_ASM102517v1_genomic.gff.gz
  File already exists: ../Data/BacDive/gff/ncbi/GCA_017874575.1_ASM1787457v1_genomic.gff.gz
  File already exist

-1

## Check missing files

In [51]:
folder1 = '../Data/BacDive/ncbi/Decompress/'
folder2 = '../Data/BacDive/gff/ncbi/Decompress/'

files1 = [os.path.splitext(f)[0][:-4] for f in os.listdir(folder1) if os.path.isfile(os.path.join(folder1, f))]
files2 = [os.path.splitext(f)[0][:-4] for f in os.listdir(folder2) if os.path.isfile(os.path.join(folder2, f))]

unique_files_in_folder1 = set(files1) - set(files2)
unique_files_in_folder2 = set(files2) - set(files1)

print(f"Nomi di file presenti solo nella cartella {folder1}:")
for filename in unique_files_in_folder1:
    print(filename)
    
print('\n\n')
print(f"Nomi di file presenti solo nella cartella {folder2}:")
for filename in unique_files_in_folder2:
    print(filename)

Nomi di file presenti solo nella cartella ../Data/BacDive/ncbi/Decompress/:
GCA_000185065.2_ASM18506v1_gen
GCA_905371965.1_SRR9217392-mag-bin.15_gen
GCA_000166715.2_ASM16671v1_gen



Nomi di file presenti solo nella cartella ../Data/BacDive/gff/ncbi/Decompress/:
GCA_000166715.1_ASM16671v1_gen
GCA_000185065.1_ASM18506v1_gen


# Generate the dataset

In [ ]:
def prepare_aflp_silco():
    os.chdir('AFLPinSilico')
    folder = '../../Data/BacDive/AFLP_perl/'

    if not os.path.exists(folder):
        os.makedirs(folder)

    # !cpan CGI -y # In case the CGI module is not installed

    if sys.platform.startswith('win'):
        perl_script = 'script_AFLP_perl.pl'
    elif sys.platform.startswith('darwin') or sys.platform.startswith('linux'):
        perl_script = 'script_AFLP_perl_UNIX.pl'
    else:
        print("Operative System not supported.")
        sys.exit(1)

    os.system(f"perl {perl_script}")

    files_name = sorted([os.path.basename(f).replace('.wri', '') for f in glob.glob(folder + '*.wri')])
    
    os.chdir('..')
    
    return files_name




def generate_dataset(files_name):
    folder = '../Data/BacDive/AFLP_perl/'
    
    aflp_int = list()
    file_col = list()

    for file in files_name:

        file_col.append(file[:13])
        aflp_int_row = list()

        aflp_50_500 = pandas.read_table(folder+file+'.wri', header=10)
        aflp_50_500.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
        aflp_50_500 = aflp_50_500.loc[(aflp_50_500['FragLength'] >= 50) & (aflp_50_500['FragLength'] <= 500)]

        freq = aflp_50_500['FragLength'].value_counts()
        frag_length = freq.sort_index()
        frag_length_index = frag_length.index

        for f in range(50,501,1):
            if f in frag_length_index:
                aflp_int_row.append(freq[f])
            else:
                aflp_int_row.append(0)

        aflp_int.append(aflp_int_row)

    aflp_int_df = pandas.DataFrame(aflp_int)
    aflp_int_df.columns = list(range(50, 501))
    aflp_int_df['sequence_genomes.Seq. accession number'] = file_col
    aflp_int_df.to_excel('../Data/BacDive/aflp_int_AFLPinSilco.xlsx',index=False)
    
    # Reads the original dataset to eliminate superfluous information, set the indexes and to do some checks.
    X = pandas.read_excel("../Data/BacDive/aflp_int_AFLPinSilco.xlsx")
    y = pandas.read_excel("../Data/BacDive/Genome_50CH_ac_0_1.xlsx")

    X['sequence_genomes.Seq. accession number'] = X['sequence_genomes.Seq. accession number'].astype(str)
    y['sequence_genomes.Seq. accession number'] = target['sequence_genomes.Seq. accession number'].astype(str) 

    X = X.rename(columns={'sequence_genomes.Seq. accession number': 'strain'}).set_index('strain').sort_index()
    y = y.rename(columns={'sequence_genomes.Seq. accession number': 'strain'}).set_index('strain').sort_index()

    y = y.iloc[:, list(range(11, 61))]
    y = y.loc[X.index]

    for i in range(len(X.index)):
        if X.index[i] != y.index[i]:
            print(f"The row indices in row {i} are not equal: {X.index[i]} vs {y.index[i]}")

    with pandas.ExcelWriter('../Data/X.xlsx') as writer:
        X.to_excel(writer, index=True)

    with pandas.ExcelWriter('../Data/Y.xlsx') as writer:
        y.to_excel(writer, index=True)
        
    print("\n\nDataset saved in Data/X.xlsx and in Data/Y.xlsx")
    
    display(X)
    display(y)

    
    
files_name = prepare_aflp_silco()
generate_dataset(files_name)

GCA_001025135.1_ASM102513v1_genomic
selectiv5 	selectiv3 
GCA_023657385.1_ASM2365738v1_genomic
selectiv5 	selectiv3 
GCA_009764365.1_L._kimchii_genomic
selectiv5 	selectiv3 
GCA_007989125.1_ASM798912v1_genomic
selectiv5 	selectiv3 
GCA_002706425.1_ASM270642v1_genomic
selectiv5 	selectiv3 
GCA_001543205.1_ASM154320v1_genomic
selectiv5 	selectiv3 
GCA_000745125.1_ASM74512v1_genomic
selectiv5 	selectiv3 
GCA_000241055.1_ASM24105v1_genomic
selectiv5 	selectiv3 
GCA_000425865.1_ASM42586v1_genomic
selectiv5 	selectiv3 
GCA_000498955.1_DSM-21115_genomic
selectiv5 	selectiv3 
GCA_900476045.1_50279_D01_genomic
selectiv5 	selectiv3 
GCA_000196555.1_ASM19655v1_genomic
selectiv5 	selectiv3 
GCA_019655995.1_ASM1965599v1_genomic
selectiv5 	selectiv3 
GCA_002907375.1_ASM290737v1_genomic
selectiv5 	selectiv3 
GCA_002259605.1_ASM225960v1_genomic
selectiv5 	selectiv3 
GCA_000024285.1_ASM2428v1_genomic
selectiv5 	selectiv3 
GCA_001039045.1_ASM103904v1_genomic
selectiv5 	selectiv3 
GCA_000426405.1_ASM4264